## 2021: Week 10 - Pokémon Hierarchies

I can hear the hesitancy already surrounding this challenge. "I know nothing about Pokémon, how can I participate?" Well, rest assured, you don't need to understand the data in order to work with it and complete this challenge. In fact, I believe this challenge could help with most hierarchical data. 

The aim of this challenge is to group Pokémon into Evolution Groups. Why? So that when we select any Pokémon, we can see all the information about what they evolved from, what they will evolve to etc. In the example below, Bulbasaur evolves into Ivysaur, who then evolves into Venusaur. Choose another Pokémon like Flareon and we can see it evolves from Eevee, who has multiple evolution possibilities.

How is this applicable to other hierarchical data? Well imagine that a Pokémon that has a null in the Evolves From field is similar to an employee at Work Level 1 in an organisation. Their manager would then sit in the Evolves To field. Our Evolution Group in this instance would be the Team the employee belongs to and it would give us the ability to type in any employee name and return information about which team they belong to and the structure of that team.

Sure, I could have chosen to design this challenge to be working with employees, but I believe it's useful to work with data outside of your comfort zone every now and then to prove to yourself that your prep skills are applicable no matter what the dataset. Plus it's just more fun to work with Pokémon!

### Input
We have 2 inputs this week:
1. Pokemon
![img](https://1.bp.blogspot.com/-6yOamFdTm1o/YCVAm0FbdtI/AAAAAAAAAvY/zQNLQy8hjAAiGj0Od3q1d1b27Mxnkml8ACLcBGAsYHQ/w400-h188/2021W10%2BInput1.png)

2. Evolution
![img2](https://1.bp.blogspot.com/-G9styqz2ZcI/YCVAaVD6E9I/AAAAAAAAAvU/O7pXXAZJsnQ3uXp2sgkSsUW8py1p6GieACLcBGAsYHQ/s320/2021W10%2BInput2.png)

### Requirment

- Input the data
- Our Pokémon dataset actually contains too many Pokémon: 
    - We're only interested in Pokémon up to Generation III, which is up to (and including) number 386
    - This means we're also not interested in mega evolutions so we can filter Pokémon whose name start with "Mega"
- Some Pokémon have more than one Type. We aren't interested in Types for this challenge so remove this field and ensure we have one row per Pokémon 
- Now we want to bring in information about what our Pokémon evolve to 
    - Warning!  In our Evolution dataset, we still have Pokémon beyond Gen III. You'll need to filter these out too, from both the evolved from and evolved to fields 
- Bring in information about what a Pokémon evolves from 
    - Ensure that we have all 386 of our Pokémon, with nulls if they don't have a pre-evolved form or if they don't evolve
- Finally, for Pokémon that have 3 evolutions, we want to know what the First Evolution is in their Evolution Group
- Some duplication may have occurred with all our joins, ensure no 2 rows are exactly the same
- Create a calculation for our Evolution Group
    - The Evolution Group will be named after the First Evolution e.g. in the above example, Bulbasaur is the name of the Evolution Group
- Output the data

### Output
![img3](https://1.bp.blogspot.com/-GXFwV1UpiX4/YCVijXraHSI/AAAAAAAAAvs/b9oxjFsWaucSvhHncE64YWynxWiBIe9tACLcBGAsYHQ/w640-h158/2021W%25600%2BOujt.png)

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Input the data

In [29]:
data = pd.read_excel("./data/Pokemon Input.xlsx", sheet_name=["Pokemon", "Evolution"], dtype="object")

In [30]:
pokemon = data["Pokemon"].copy()
evolution = data["Evolution"].copy()

### Our Pokémon dataset actually contains too many Pokémon:
 - We're only interested in Pokémon up to Generation III, which is up to (and including) number 386
 - This means we're also not interested in mega evolutions so we can filter Pokémon whose name start with "Mega

In [31]:
pokemon = pokemon.rename(columns={"#": "Number"})
pokemon.Number = pokemon.Number.astype(float).astype(int)
pokemon = pokemon.sort_values(by="Number")
pokemon = pokemon.loc[(pokemon["Number"] <= 386)]

mega = pokemon["Name"].map(lambda x: x.startswith("Mega"))
pokemon = pokemon.loc[~mega, :]
pokemon.shape

(570, 10)

### Some Pokémon have more than one Type. We aren't interested in Types for this challenge so remove this field and ensure we have one row per Pokémon

In [32]:
pokemon = pokemon.drop_duplicates(subset=["Number"])
pokemon.shape

(385, 10)

### Now we want to bring in information about what our Pokémon evolve to
- Warning!  In our Evolution dataset, we still have Pokémon beyond Gen III. 
- You'll need to filter these out too, from both the evolved from and evolved to fields

In [33]:
evolution = evolution.loc[(evolution["Evolving from"].isin(pokemon["Name"])) | 
                          (evolution["Evolving to"].isin(pokemon["Name"]))]
evolution = evolution.drop_duplicates()
evolution.head()

,Evolving from,Evolving to,Level,Condition,Evolution Type
0,Bulbasaur,Ivysaur,16,NaN,Level
1,Ivysaur,Venusaur,32,NaN,Level
2,Charmander,Charmeleon,16,NaN,Level
3,Charmeleon,Charizard,36,NaN,Level
4,Squirtle,Wartortle,16,NaN,Level


### Bring in information about what a Pokémon evolves from
- Ensure that we have all 386 of our Pokémon, with nulls if they don't have a pre-evolved form or if they don't evolve

In [34]:
pokemon = pokemon.merge(evolution, how="left", left_on="Name", right_on="Evolving from")
pokemon.shape

(398, 15)

### Finally, for Pokémon that have 3 evolutions, we want to know what the First Evolution is in their Evolution Group
- Some duplication may have occurred with all our joins, ensure no 2 rows are exactly the same


In [35]:
pokemon = pokemon.drop_duplicates()
pokemon.shape

(398, 15)

### Create a calculation for our Evolution Group
- The Evolution Group will be named after the First Evolution e.g. in the above example, Bulbasaur is the name of the Evolution Group

In [36]:
# Evolving group calculation (compare 3 rows in if statement)
pokemon["Evolution Group"] = pokemon["Name"]

for i in range(len(pokemon) - 2):
    if pokemon.loc[i, "Evolving to"] == pokemon.loc[i+1, "Name"] and pokemon.loc[i+1, "Evolving to"] == pokemon.loc[i+2, "Name"] :
        pokemon.loc[[i, i+1, i+2], "Evolution Group"] = pokemon.loc[i, "Name"]
    else:
        pass

pokemon.head()

,Number,Name,Type,Total,HP,Attack,Defense,Special Attack,Special Defense,Speed,Evolving from,Evolving to,Level,Condition,Evolution Type,Evolution Group
0,1,Bulbasaur,GRASS,318,45,49,49,65,65,45,Bulbasaur,Ivysaur,16,NaN,Level,Bulbasaur
1,2,Ivysaur,GRASS,405,60,62,63,80,80,60,Ivysaur,Venusaur,32,NaN,Level,Bulbasaur
2,3,Venusaur,GRASS,525,80,82,83,100,100,80,NaN,NaN,NaN,NaN,NaN,Bulbasaur
3,4,Charmander,FIRE,309,39,52,43,60,50,65,Charmander,Charmeleon,16,NaN,Level,Charmander
4,5,Charmeleon,FIRE,405,58,64,58,80,65,80,Charmeleon,Charizard,36,NaN,Level,Charmander


In [37]:
pokemon = pokemon.loc[:, ["Evolution Group", "Number", "Name", "Total", "HP", "Attack", "Defense", "Special Attack",
                          "Special Defense", "Speed", "Evolving from", "Evolving to", "Level", "Condition", "Evolution Type"]]
pokemon.head()

,Evolution Group,Number,Name,Total,HP,Attack,Defense,Special Attack,Special Defense,Speed,Evolving from,Evolving to,Level,Condition,Evolution Type
0,Bulbasaur,1,Bulbasaur,318,45,49,49,65,65,45,Bulbasaur,Ivysaur,16,NaN,Level
1,Bulbasaur,2,Ivysaur,405,60,62,63,80,80,60,Ivysaur,Venusaur,32,NaN,Level
2,Bulbasaur,3,Venusaur,525,80,82,83,100,100,80,NaN,NaN,NaN,NaN,NaN
3,Charmander,4,Charmander,309,39,52,43,60,50,65,Charmander,Charmeleon,16,NaN,Level
4,Charmander,5,Charmeleon,405,58,64,58,80,65,80,Charmeleon,Charizard,36,NaN,Level


In [38]:
pokemon.to_csv("./output/Week10_output.csv")